<a href="https://colab.research.google.com/github/georgezero/rsna25-deid-using-chatgpt-llms/blob/main/05_synthetic_report_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synthetic Radiology Report Generator

**The problem:** You need realistic radiology reports for testing, training, or demonstrations, but you can't use real patient data.

**The solution:** Use LLMs to generate completely synthetic but clinically realistic reports with fake PHI.

In this notebook, you'll learn to:
1. Generate single realistic reports with customizable parameters
2. Create matched pairs (PHI version + deidentified version)
3. Batch generate diverse report datasets
4. Control clinical scenarios, complexity, and PHI types



## Quick start
- Run the `pip install` cell.
- Get a free API key from [Google AI Studio](https://aistudio.google.com/app/apikey).
- Set your `GOOGLE_API_KEY` in the code below.

In [1]:
# Install dependencies
%pip install -q -U google-generativeai pandas

You will need to create an API key. Click the link for a quick tutorial: [Tutorial](https://drive.google.com/file/d/1fGoSsapol9JY9oqu-UulLJfMm-v2egWC/view?usp=drive_link)

In [2]:
import os
import json
import pandas as pd
import google.generativeai as genai
from IPython.display import display, Markdown

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or "PASTE_YOUR_KEY_HERE"

if GOOGLE_API_KEY == "PASTE_YOUR_KEY_HERE":
    print("⚠️ Please paste your Google API Key in the variable above.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Gemini API configured")

model = genai.GenerativeModel('models/gemini-2.0-flash-lite')

✅ Gemini API configured


## 1️⃣ Basic Report Generation

Let's start simple - generate a single realistic radiology report.

In [4]:
basic_prompt = """
Generate a realistic radiology report for a CT scan of the chest.

Include:
- Realistic (but completely fake) patient demographics
- A plausible clinical history
- Detailed findings (some normal, some abnormal)
- Professional impression section
- Fake but realistic PHI (names, dates, MRN, physician names)

Make it read like a real clinical report from a busy radiology department.
The findings should be medically accurate and consistent.
"""

response = model.generate_content(basic_prompt)
print("GENERATED REPORT:")
print("=" * 60)
print(response.text)

GENERATED REPORT:
## Radiology Report

**Patient Information:**

*   **Patient Name:** Anya Petrova
*   **Medical Record Number (MRN):** 876-530-999
*   **Date of Birth:** 03/12/1972
*   **Gender:** Female
*   **Referring Physician:** Dr. Eleanor Vance, MD
*   **Date of Exam:** 10/26/2023
*   **Date of Report:** 10/26/2023

**Clinical History:**

65-year-old female presents with a 2-week history of worsening cough, productive of minimal white sputum, and associated left-sided chest pain exacerbated by deep inspiration. Denies fever, chills, or weight loss. Previous history of treated hypertension and well-controlled asthma. Concerned for possible pneumonia or pulmonary embolism.

**Exam:**

CT Chest with IV contrast was performed on a GE Revolution CT scanner. Images were obtained with 2.5mm slice thickness and reconstructed in both axial, coronal, and sagittal planes.

**Findings:**

*   **Lungs:**
    *   Mild bronchial wall thickening is noted in the bilateral lower lobes, consisten

## Parameterized Generation

Control exactly what kind of report you want to generate.

In [5]:
def generate_report(
    modality="CT",
    body_part="chest",
    clinical_scenario="routine screening",
    findings_complexity="moderate",  # simple, moderate, complex
    include_phi=True,
    patient_age_range="adult",  # pediatric, adult, geriatric
    urgency="routine"  # routine, urgent, emergent
):
    """
    Generate a synthetic radiology report with specified parameters.
    """
    prompt = f"""
Generate a realistic radiology report with these specifications:

MODALITY: {modality}
BODY PART: {body_part}
CLINICAL SCENARIO: {clinical_scenario}
FINDINGS COMPLEXITY: {findings_complexity} (number and severity of findings)
PATIENT AGE RANGE: {patient_age_range}
URGENCY LEVEL: {urgency}
INCLUDE PHI: {"Yes - include realistic fake names, dates, MRNs, physician names" if include_phi else "No - use placeholders like [NAME], [DATE], [MRN]"}

Requirements:
1. Follow standard radiology report format (Exam, History, Technique, Comparison, Findings, Impression)
2. Make findings medically accurate and internally consistent
3. Match the urgency level in your language and recommendations
4. Include appropriate comparison studies if relevant
5. Add realistic technical details for the modality

Generate only the report text, no additional commentary.
"""
    response = model.generate_content(prompt)
    return response.text

In [6]:
# Example 1: Simple chest X-ray
report1 = generate_report(
    modality="X-ray",
    body_part="chest",
    clinical_scenario="pre-operative clearance for knee surgery",
    findings_complexity="simple",
    patient_age_range="adult"
)
print("SIMPLE CHEST X-RAY:")
print("=" * 60)
print(report1)

SIMPLE CHEST X-RAY:
**RADIOLOGY REPORT**

**PATIENT NAME:** Jane Doe
**MRN:** 1234567
**DATE OF EXAM:** 2024-10-27
**EXAM:** Chest X-ray, PA and Lateral

**REFERRING PHYSICIAN:** Dr. Robert Smith

**HISTORY:** Pre-operative clearance for knee surgery.

**TECHNIQUE:**  PA and lateral chest radiographs were obtained with the patient in the upright position. The PA view was obtained at 72 inches source-to-image receptor distance (SID). Appropriate collimation was utilized, and the images were reviewed for adequate inspiration and absence of rotation.

**COMPARISON:**  None available.

**FINDINGS:**
The lungs are clear, with no focal consolidation, pneumothorax, or pleural effusion identified.  The pulmonary vasculature is within normal limits. The cardiac silhouette is normal in size and configuration. The mediastinum is unremarkable.  The bony structures of the chest appear intact.

**IMPRESSION:**
Normal chest X-ray. Findings are consistent with pre-operative evaluation.  No acute cardi

In [7]:
# Example 2: Complex emergency CT
report2 = generate_report(
    modality="CT",
    body_part="abdomen and pelvis",
    clinical_scenario="acute abdominal pain, fever, elevated WBC",
    findings_complexity="complex",
    patient_age_range="geriatric",
    urgency="emergent"
)
print("COMPLEX EMERGENCY CT:")
print("=" * 60)
print(report2)

COMPLEX EMERGENCY CT:
**MEDICAL IMAGING REPORT**

**PATIENT NAME:** Robert Abernathy
**MRN:** 1234567
**DATE OF EXAM:** 2024-02-29
**DATE OF REPORT:** 2024-02-29
**EXAM:** CT Abdomen and Pelvis with IV Contrast
**REFERRING PHYSICIAN:** Dr. Eleanor Vance
**PERFORMING PHYSICIAN:** Dr. Marcus Chen

**HISTORY:** Acute abdominal pain, fever, and elevated WBC. Possible diagnosis of acute abdomen.

**TECHNIQUE:**
Axial CT images of the abdomen and pelvis were obtained following the intravenous administration of 100 mL of Isovue 370 contrast. Pre-contrast images were obtained. Slice thickness: 3 mm. Multiplanar reformats were performed in the sagittal and coronal planes.

**COMPARISON:**
None available.

**FINDINGS:**

Liver: Mildly heterogeneous attenuation. No focal hepatic lesions identified.
Gallbladder: The gallbladder is visualized. No evidence of cholelithiasis or cholecystitis.
Biliary Tree: Normal caliber.
Pancreas: The pancreas appears unremarkable.
Spleen: Normal size and appearance

In [8]:
# Example 3: Pediatric MRI
report3 = generate_report(
    modality="MRI",
    body_part="brain",
    clinical_scenario="new onset seizures",
    findings_complexity="moderate",
    patient_age_range="pediatric",
    urgency="urgent"
)
print("PEDIATRIC BRAIN MRI:")
print("=" * 60)
print(report3)

PEDIATRIC BRAIN MRI:
**EXAM:** MRI Brain

**HISTORY:** New onset seizures.  Patient: Emily Carter, DOB: 05/15/2018, MRN: 1234567.

**TECHNIQUE:**  MRI of the brain was performed on a 3T scanner utilizing a dedicated 16-channel head coil.  Imaging was obtained in the axial, sagittal, and coronal planes. Sequences included: T1-weighted pre- and post-contrast, T2-weighted, FLAIR, DWI and ADC sequences.  Slice thickness was 3mm for all sequences.  IV Gadolinium contrast (Gadobutrol) was administered at a dose of 0.1 mmol/kg.

**COMPARISON:** None.

**FINDINGS:**

There is a 1.8 x 1.5 x 1.2 cm enhancing lesion in the right temporal lobe, centered within the mesial temporal structures. The lesion demonstrates heterogeneous enhancement following contrast administration. Surrounding the lesion is mild T2 and FLAIR hyperintensity, suggestive of edema. There is associated mass effect, with mild effacement of the ipsilateral temporal horn of the lateral ventricle and slight midline shift to the l

## Generate Matched Pairs (PHI + Deidentified)

This is useful for training and testing deidentification systems.
Generate a report with PHI, then its properly deidentified version.

In [9]:
def generate_matched_pair(
    modality="CT",
    body_part="chest",
    clinical_scenario="shortness of breath"
):
    """
    Generate a report with PHI and its deidentified version.
    Returns both versions plus a mapping of what was changed.
    """
    prompt = f"""
Generate a matched pair of radiology reports - one with PHI and one deidentified.

MODALITY: {modality}
BODY PART: {body_part}
CLINICAL SCENARIO: {clinical_scenario}

Provide your response in this EXACT JSON format:
{{
  "original_with_phi": "[Full report with realistic fake PHI]",
  "deidentified": "[Same report with PHI properly removed/replaced]",
  "phi_mapping": [
    {{"type": "Patient Name", "original": "John Smith", "replacement": "[PATIENT NAME]"}},
    {{"type": "MRN", "original": "12345678", "replacement": "[MRN]"}},
    ... (list all PHI elements)
  ]
}}

Make sure:
1. The clinical content is IDENTICAL between versions
2. Only PHI elements are changed
3. Include diverse PHI types (names, dates, IDs, locations, etc.)
4. The deidentification follows HIPAA Safe Harbor guidelines

Return ONLY valid JSON, no other text.
"""
    response = model.generate_content(prompt)

    # Parse the JSON response
    try:
        # Clean up the response (remove markdown code blocks if present)
        text = response.text.strip()
        if text.startswith("```json"):
            text = text[7:]
        if text.startswith("```"):
            text = text[3:]
        if text.endswith("```"):
            text = text[:-3]

        result = json.loads(text.strip())
        return result
    except json.JSONDecodeError as e:
        print(f"Warning: Could not parse JSON response. Error: {e}")
        print("Raw response:")
        print(response.text)
        return None

In [10]:
# Generate a matched pair
pair = generate_matched_pair(
    modality="MRI",
    body_part="lumbar spine",
    clinical_scenario="chronic low back pain with radiculopathy"
)

if pair:
    print("ORIGINAL REPORT (WITH PHI):")
    print("=" * 60)
    print(pair["original_with_phi"])
    print("\n")
    print("DEIDENTIFIED VERSION:")
    print("=" * 60)
    print(pair["deidentified"])
    print("\n")
    print("PHI MAPPING:")
    print("=" * 60)
    for item in pair["phi_mapping"]:
        print(f"  {item['type']}: '{item['original']}' → '{item['replacement']}'")

ORIGINAL REPORT (WITH PHI):
PATIENT NAME: John Smith
MRN: 12345678
DATE OF EXAM: 2024-03-08
DATE OF BIRTH: 1970-05-15
REFERRING PHYSICIAN: Dr. Emily Carter, Spine Clinic of Anytown

EXAMINATION: MRI lumbar spine.

CLINICAL INDICATION: Chronic low back pain with radiculopathy.

TECHNIQUE: Sagittal T1-weighted, T2-weighted, and STIR images; axial T2-weighted images were obtained.

FINDINGS: 

Alignment: Normal lumbar spine alignment.

Vertebral Bodies: Mild endplate degenerative changes are noted at L4-L5 and L5-S1. No fracture or significant bone marrow edema is identified.

Intervertebral Discs: 

*   L4-L5: There is a broad-based disc bulge with mild central canal stenosis. There is also mild bilateral foraminal narrowing.
*   L5-S1: There is a small disc herniation, slightly left-sided, contacting the left S1 nerve root. Mild central canal stenosis is present.

Spinal Cord and Cauda Equina: The spinal cord and cauda equina are unremarkable.

Facet Joints: Mild facet arthropathy is no

## Batch Generation: Create a Dataset

Generate multiple diverse reports at once for training datasets.

In [11]:
# Define the variety of reports we want
report_configs = [
    {"modality": "X-ray", "body_part": "chest", "scenario": "cough and fever", "complexity": "simple"},
    {"modality": "CT", "body_part": "head", "scenario": "trauma, fall from standing", "complexity": "moderate"},
    {"modality": "CT", "body_part": "chest", "scenario": "lung cancer staging", "complexity": "complex"},
    {"modality": "MRI", "body_part": "knee", "scenario": "sports injury, ACL concern", "complexity": "moderate"},
    {"modality": "Ultrasound", "body_part": "abdomen", "scenario": "right upper quadrant pain", "complexity": "simple"},
    {"modality": "CT", "body_part": "abdomen pelvis", "scenario": "appendicitis workup", "complexity": "moderate"},
    {"modality": "MRI", "body_part": "brain", "scenario": "headaches and vision changes", "complexity": "complex"},
    {"modality": "X-ray", "body_part": "lumbar spine", "scenario": "chronic back pain", "complexity": "simple"},
]

print(f"Configured {len(report_configs)} report types to generate")

Configured 8 report types to generate


In [12]:
# Generate the batch (this will take a minute, you might run out of ressources of the free tier)
generated_reports = []

print("Generating reports...")
for i, config in enumerate(report_configs):
    print(f"  [{i+1}/{len(report_configs)}] {config['modality']} {config['body_part']}...", end=" ")

    report = generate_report(
        modality=config["modality"],
        body_part=config["body_part"],
        clinical_scenario=config["scenario"],
        findings_complexity=config["complexity"]
    )

    generated_reports.append({
        "id": i + 1,
        "modality": config["modality"],
        "body_part": config["body_part"],
        "scenario": config["scenario"],
        "complexity": config["complexity"],
        "report_text": report
    })
    print("✓")

print(f"\nGenerated {len(generated_reports)} reports!")

Generating reports...
  [1/8] X-ray chest... ✓
  [2/8] CT head... ✓
  [3/8] CT chest... ✓
  [4/8] MRI knee... ✓
  [5/8] Ultrasound abdomen... ✓
  [6/8] CT abdomen pelvis... ✓
  [7/8] MRI brain... ✓
  [8/8] X-ray lumbar spine... ✓

Generated 8 reports!


In [13]:
# View as a summary table
summary_df = pd.DataFrame([
    {
        "ID": r["id"],
        "Modality": r["modality"],
        "Body Part": r["body_part"],
        "Scenario": r["scenario"],
        "Complexity": r["complexity"],
        "Length (chars)": len(r["report_text"])
    }
    for r in generated_reports
])

display(summary_df)

,ID,Modality,Body Part,Scenario,Complexity,Length (chars)
0,1,X-ray,chest,cough and fever,simple,950
1,2,CT,head,"trauma, fall from standing",moderate,1472
2,3,CT,chest,lung cancer staging,complex,2331
3,4,MRI,knee,"sports injury, ACL concern",moderate,1698
4,5,Ultrasound,abdomen,right upper quadrant pain,simple,1899
5,6,CT,abdomen pelvis,appendicitis workup,moderate,1555
6,7,MRI,brain,headaches and vision changes,complex,2126
7,8,X-ray,lumbar spine,chronic back pain,simple,1118


In [14]:
# View a specific report from the batch
report_index = 2  # Change this to view different reports (0-7)

selected = generated_reports[report_index]
print(f"📄 REPORT #{selected['id']}: {selected['modality']} {selected['body_part'].upper()}")
print(f"   Scenario: {selected['scenario']}")
print(f"   Complexity: {selected['complexity']}")
print("=" * 60)
print(selected['report_text'])

📄 REPORT #3: CT CHEST
   Scenario: lung cancer staging
   Complexity: complex
**Exam:** CT Chest with IV Contrast

**Patient Name:** John Smith
**MRN:** 1234567
**DOB:** 01/01/1960
**Date of Exam:** 10/27/2023
**Referring Physician:** Dr. Emily Carter

**History:** Staging for known non-small cell lung cancer.

**Technique:** Axial CT images of the chest were obtained from the lung apices through the adrenal glands following intravenous administration of 100 mL of Isovue 370 contrast. Image acquisition parameters: 120 kVp, automated tube current modulation, slice thickness 3mm reconstructed to 1 mm. Coronal and sagittal reconstructions were performed.

**Comparison:** Prior CT chest dated 06/15/2023.

**Findings:**

Lungs:
*   There is a 4.8 cm lobulated mass in the right upper lobe, consistent with the previously identified primary lung malignancy. Interval growth is noted. The mass demonstrates heterogeneous enhancement post-contrast. Associated pleural retraction is present.
*   Mul

## Export Your Dataset

Save the generated reports for use in other projects.

In [15]:
# Export as JSON
output_filename = "synthetic_radiology_reports.json"

with open(output_filename, 'w') as f:
    json.dump(generated_reports, f, indent=2)

print(f"✅ Exported {len(generated_reports)} reports to {output_filename}")

# Also create a CSV with just metadata
summary_df.to_csv("synthetic_reports_metadata.csv", index=False)
print(f"✅ Exported metadata to synthetic_reports_metadata.csv")

✅ Exported 8 reports to synthetic_radiology_reports.json
✅ Exported metadata to synthetic_reports_metadata.csv


In [16]:
# Download the files (in Colab)
try:
    from google.colab import files
    files.download(output_filename)
    files.download("synthetic_reports_metadata.csv")
except ImportError:
    print("Not running in Colab - files saved to current directory")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Specialized Generators

Generate reports with specific characteristics for targeted testing.

In [17]:
def generate_edge_case_report(edge_case_type):
    """
    Generate reports with specific edge cases for testing deidentification.
    """
    edge_case_prompts = {
        "multiple_dates": """
            Generate a radiology report that contains MANY different dates:
            - Exam date
            - Multiple comparison study dates
            - Date of prior surgery mentioned in history
            - Date of symptom onset
            - Follow-up recommendation date
            Include at least 6 different dates in various formats (MM/DD/YYYY, Month DD, YYYY, etc.)
        """,

        "many_names": """
            Generate a radiology report that contains MANY different names:
            - Patient name
            - Referring physician
            - Radiologist
            - Technologist
            - Previous physician mentioned in history
            - Family member mentioned ("patient's daughter reports...")
            Include at least 5 different people's names.
        """,

        "embedded_phi_in_findings": """
            Generate a radiology report where PHI is embedded within the clinical findings:
            - Mention the patient's employer in the history
            - Reference a specific geographic location
            - Include a device with a serial number
            - Mention a specific hospital or clinic by name
            - Reference a family member's medical history
        """,

        "international_names": """
            Generate a radiology report with international/diverse names:
            - Use names from different cultural backgrounds
            - Include names with special characters or diacritics
            - Use names that might be challenging for simple regex patterns
        """,

        "numeric_identifiers": """
            Generate a radiology report heavy on numeric identifiers:
            - Multiple MRN formats
            - Accession numbers
            - Phone numbers in history
            - Insurance ID
            - Room numbers
            - Device serial numbers
        """
    }

    base_prompt = edge_case_prompts.get(edge_case_type, edge_case_prompts["multiple_dates"])

    full_prompt = f"""
    {base_prompt}

    Create a complete, realistic radiology report (CT or MRI) with all standard sections.
    Make sure the clinical content is medically accurate.
    Generate only the report text.
    """

    response = model.generate_content(full_prompt)
    return response.text

In [18]:
# Generate an edge case report with many dates
edge_report = generate_edge_case_report("multiple_dates")
print("EDGE CASE: Multiple Dates")
print("=" * 60)
print(edge_report)

EDGE CASE: Multiple Dates
**Patient Name:** John Doe
**Patient Medical Record Number:** 1234567
**Exam Date:** 07/26/2024
**Referring Physician:** Dr. Jane Smith
**Study Type:** MRI of the Lumbar Spine

**Clinical Indication:** Chronic low back pain, possible disc herniation.

**Comparison:**
*   MRI Lumbar Spine, Month March 15, 2023
*   CT Lumbar Spine, 01/20/2022

**History:** Patient presents with chronic low back pain for approximately six months. Symptoms started around January 10, 2024. The patient underwent a laminectomy at L4-L5 on 05/01/2021.

**Technique:** Multiplanar, multi-sequence MRI of the lumbar spine was performed using a 3.0T magnet, including sagittal T1-weighted, T2-weighted, and STIR sequences, as well as axial T2-weighted and T1-weighted sequences.

**Findings:**

*   **Vertebral Bodies:** There is mild degenerative endplate change at multiple lumbar levels. No significant vertebral body fracture or significant marrow edema is identified.
*   **Intervertebral Di

In [19]:
# Generate an edge case with embedded PHI in findings
edge_report_2 = generate_edge_case_report("embedded_phi_in_findings")
print("EDGE CASE: Embedded PHI in Clinical Content")
print("=" * 60)
print(edge_report_2)

EDGE CASE: Embedded PHI in Clinical Content
**Report:**

**Patient Name:** Jane Doe
**Patient MRN:** 1234567
**Date of Exam:** 2024-10-27
**Referring Physician:** Dr. Smith
**Exam Type:** MRI Brain without contrast
**Clinical Indication:** Headache, rule out mass. History of similar headaches. Patient reports these headaches have been particularly concerning, especially since her recent promotion at **Acme Corp** and increased workload.

**History:** The patient is a 45-year-old female presenting with a three-month history of intermittent headaches. These headaches are described as throbbing, located primarily in the frontal region, and associated with nausea. She reports no recent head trauma. Her father had a history of brain aneurysms and the patient is concerned given her family medical history. The patient reports she lives and works in **the heart of downtown Chicago**, and the stress of her new position has been significant.

**Technique:** Sagittal T1-weighted, axial T2-weighte

## Interactive: Build Your Own Report

Customize all parameters and generate exactly the report you need.

In [20]:
# ===== CUSTOMIZE THESE PARAMETERS =====

my_modality = "CT"  # Options: X-ray, CT, MRI, Ultrasound, PET, Nuclear Medicine
my_body_part = "chest"  # Examples: chest, abdomen, brain, spine, knee, shoulder
my_scenario = "lung nodule follow-up"  # Describe the clinical situation
my_complexity = "moderate"  # Options: simple, moderate, complex
my_age_group = "adult"  # Options: pediatric, adult, geriatric
my_urgency = "routine"  # Options: routine, urgent, emergent

# ===== GENERATE =====

custom_report = generate_report(
    modality=my_modality,
    body_part=my_body_part,
    clinical_scenario=my_scenario,
    findings_complexity=my_complexity,
    patient_age_range=my_age_group,
    urgency=my_urgency
)

print(f"YOUR CUSTOM REPORT: {my_modality} {my_body_part.upper()}")
print("=" * 60)
print(custom_report)

YOUR CUSTOM REPORT: CT CHEST
**EXAM:** Chest CT

**HISTORY:** Follow-up for previously identified right upper lobe lung nodule. John Smith, MRN: 1234567, DOB: 01/15/1960.

**TECHNIQUE:** Axial CT images of the chest were obtained from the lung apices to the lung bases following intravenous administration of 100 mL of Isovue 370 contrast. Scan parameters: 120 kVp, automated mA, 3 mm slice thickness reconstructed to 1 mm slices for lung windows and soft tissue windows.

**COMPARISON:** Chest CT dated 06/15/2023.

**FINDINGS:**

Compared to the prior study, the right upper lobe nodule demonstrates slight interval growth. It now measures approximately 9 mm in greatest dimension, compared to 7 mm on the prior examination. The nodule remains well-circumscribed with no associated spiculations or pleural retraction. No new nodules are identified.

Stable mild emphysema is noted in the upper lobes. No significant mediastinal or hilar lymphadenopathy is present. The trachea and main bronchi are 

## Key Takeaways

1. LLMs can generate realistic synthetic data

2. Parameterized generation gives control

3. Having PHI and deidentified versions together is essential for training/testing.

4. Generate specific challenging scenarios to stress-test your systems.

5. Generate hundreds of reports in minutes for dataset creation.

### Limitations
- Generated reports may have subtle clinical inaccuracies
- Should not be used for actual clinical training without expert review
- LLM outputs can vary - always validate critical use cases